In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
cmap = matplotlib.colormaps.get('tab10').colors
import torch
import torch_geometric as tg
from gen_autoencoder_dataset import AutoencoderDataset

In [62]:
n = 30
dataset = AutoencoderDataset(root='autoencoder_graphs', n_graphs_per_type=n)
dataset.process()
train_loader = tg.loader.DataLoader(dataset[:n*12], batch_size=16, shuffle=True)
test_loader = tg.loader.DataLoader(dataset[n*12:], batch_size=16, shuffle=True)
train_loader.dataset[2]
device= 'cuda' if torch.cuda.is_available() else 'cpu'


  0%|          | 0/420 [00:00<?, ?it/s]

100%|██████████| 420/420 [00:02<00:00, 142.88it/s]


## New idea: Do on my own

In [63]:
num_features = dataset.num_features
alpha = 0.5 # weight of the attribute loss


class dominant(torch.nn.Module):
    def __init__(self):
        super(dominant, self).__init__()
        self.encoder = torch.nn.ModuleList()
        self.encoder.append(tg.nn.GCNConv(num_features, 3))
        self.encoder.append(tg.nn.GCNConv(3, 2))
        
        
        self.attr_decoder =torch.nn.ModuleList()
        self.attr_decoder.append(tg.nn.GCNConv(2, 3))
        self.attr_decoder.append(tg.nn.GCNConv(3, num_features))
    def forward(self, x, edge_index):
        # encoder
        #print(x.shape, edge_index.shape)
        z = self.encoder[0](x, edge_index).relu()
        z = self.encoder[1](z, edge_index).relu()
        # decoder
        adj = torch.sigmoid(z @ z.T) # structure decoder
        
        x_ = self.attr_decoder[0](z, edge_index).relu()
        x_ = self.attr_decoder[1](x_, edge_index).relu() # attribute decoder
        return x_, adj
    
        
        
model = dominant()
model.to(device)
print(model)

def loss_func(x, x_, adj, adj_):
    # print(f"x shape: {x.shape}, x_ shape: {x_.shape}")
    # print(f"edge_index shape: {edge_index.shape}, adj shape: {adj.shape}")
    #edge_index = edge_index.type(torch.float)
    adj = adj.type(torch.float)
    return alpha * torch.nn.functional.mse_loss(x, x_) + (1-alpha) * torch.nn.functional.mse_loss(adj, adj_)

optimizer = torch.optim.Adam(model.parameters())

dominant(
  (encoder): ModuleList(
    (0): GCNConv(3, 3)
    (1): GCNConv(3, 2)
  )
  (attr_decoder): ModuleList(
    (0): GCNConv(2, 3)
    (1): GCNConv(3, 3)
  )
)


In [64]:
# training:
epochs = 10
def train(loader):
    model.train()
    avg_loss = 0
    for data in loader:
        data = data.to(device)
        x_, adj = model.forward(data.x, data.edge_index)
    
        loss = loss_func(data.x, x_, tg.utils.to_dense_adj(data.edge_index)[0], adj)
        avg_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return avg_loss / len(loader)
for epoch in range(1, epochs+1):
    loss = train(train_loader)
    print('Epoch: {:03d}, Training Loss: {:.4f}'.format(epoch, loss))

Epoch: 001, Training Loss: 8.2346
Epoch: 002, Training Loss: 8.1124
Epoch: 003, Training Loss: 7.9121
Epoch: 004, Training Loss: 7.2183
Epoch: 005, Training Loss: 6.4234
Epoch: 006, Training Loss: 5.7183
Epoch: 007, Training Loss: 5.1464
Epoch: 008, Training Loss: 4.9318
Epoch: 009, Training Loss: 4.7343
Epoch: 010, Training Loss: 4.5434
